In [1]:
from abc import ABC, abstractmethod
from numerapi import CryptoAPI
import pandas as pd
import numpy as np
from datetime import datetime, timezone

import json
import os
import requests
from websocket import create_connection

import eth_account
from eth_account.signers.local import LocalAccount

from hyperliquid.exchange import Exchange
from hyperliquid.info import Info
from hyperliquid.utils.constants import MAINNET_API_URL, TESTNET_API_URL

from oracle_interfaces import OracleInterface, NumeraiTBNOracle
from dex_interfaces import DEXInterface, HyperLiquidDEX


# Portfolio Manager
class PortfolioManager:
    def __init__(self, oracle: OracleInterface, dex: DEXInterface):
        """Initialize the Portfolio Manager with an Oracle implementation."""
        self.oracle = oracle
        self.dex = dex

    def manage_portfolio(self, timestamp = datetime.now(timezone.utc) ):
        """Fetch weights from the Oracle and print them."""
        print(f"[PortfolioManager] Requesting portfolio weights for {timestamp}.")
        tradable_universe = dex.get_universe()
        # tradable_universe = ['BTC','ETH','AAVE','LDO','SUI','WLD','GOAT','EIGEN','AVAX','ENS']
        weights = self.oracle.fetch_portfolio_weights(
            timestamp,
            tradable_universe,
            lags=30
        )

        if not self.oracle.validate_weights(weights):
            raise ValueError("Invalid portfolio weights received from Oracle.")

        print(f"[PortfolioManager] Portfolio weights: {weights}")

        dex.set_portfolio_weights( weights )




In [6]:
# Instantiate Oracle and DEX interfaces
oracle = NumeraiTBNOracle(tb=30)
dex = HyperLiquidDEX("testnet")

2024-12-18 15:49:24,700 INFO numerapi.utils: target file already exists
2024-12-18 15:49:24,702 INFO numerapi.utils: download complete


[DEX] Universe Set Successfully.
[DEX] Running with account address: 0x39609221106F50DD140011e2F11cEE2338dD12dB


2024-12-18 15:49:25,915 INFO websocket: Websocket connected


[DEX] Getting Positions.
[DEX] SOL: 18.7416
[DEX] BTC: 18.1917
[DEX] ETH: 18.463
[DEX] BNB: 18.76446
[DEX] AVAX: 18.46698
[DEX] APE: -18.29909
[DEX] OP: 18.40524
[DEX] WLD: -18.3525
[DEX] MKR: 18.4525
[DEX] AAVE: 17.5285
[DEX] SUI: -18.46908
[DEX] INJ: 17.4468
[DEX] BLZ: 18.39363
[DEX] BANANA: -17.3644
[DEX] BIGTIME: -18.36674
[DEX] KAS: 18.03249
[DEX] BLUR: -18.45988
[DEX] TIA: -18.21064
[DEX] BSV: 18.1872
[DEX] TON: 53.987
[DEX] ADA: -18.69961
[DEX] MINA: 18.13786
[DEX] GAS: 18.48171
[DEX] FET: 19.0203
[DEX] NEAR: 18.06835
[DEX] ORDI: -18.27016
[DEX] BADGER: -18.53766
[DEX] PYTH: -18.50534
[DEX] RUNE: 18.56435
[DEX] SUSHI: -18.4626
[DEX] ILV: -18.24498
[DEX] MAV: -18.29569
[DEX] IMX: 18.21255
[DEX] SUPER: 18.9343
[DEX] GALA: -18.380914
[DEX] ACE: -18.320531
[DEX] CAKE: -18.29295
[DEX] ENS: -18.3103
[DEX] ETC: 18.27987
[DEX] UMA: 18.28414
[DEX] REQ: -18.3456
[DEX] AR: 18.40212
[DEX] HBAR: -18.5427
[DEX] LISTA: 18.23878
[DEX] POL: -18.23325
[DEX] XLM: -53.6544
[DEX] SAND: -18.621
[DEX]

In [8]:
print(dex.net_liq)

1105.344524


In [4]:
print( dex.positions )

{'BTC': 56.41048, 'ETH': 55.266, 'BNB': 56.94326, 'AVAX': 18.459, 'APE': -18.38814, 'AAVE': 17.6095, 'INJ': 52.437, 'TON': 53.932, 'ADA': -18.7454, 'SUSHI': -52.40556, 'ACE': -18.388345, 'AR': 18.44271, 'HBAR': -59.76997, 'XLM': -53.76105, 'SAND': -53.5135, 'ALGO': -18.3995, 'IOTA': -52.11615}


In [5]:
oracle.mm['date'].min()

Timestamp('2024-05-07 00:00:00')

In [9]:
# Instantiate Portfolio Manager with Oracle
portfolio_manager = PortfolioManager(oracle, dex)

# Perform portfolio management
portfolio_manager.manage_portfolio()

[PortfolioManager] Requesting portfolio weights for 2024-12-18 20:47:00.218892+00:00.
[Oracle] Computing portfolio weights for timestamp 2024-12-18 20:47:00.218892+00:00.
[Oracle] Portfolio Date: 2024-12-17 00:00:00
[PortfolioManager] Portfolio weights: {'AAVE': 0.016666666666666666, 'ACE': -0.016666666666666666, 'ADA': -0.016666666666666666, 'ALGO': -0.016666666666666666, 'APE': -0.016666666666666666, 'APT': 0.0, 'AR': 0.016666666666666666, 'ARB': 0.0, 'ATOM': 0.0, 'AVAX': 0.016666666666666666, 'AXL': 0.016666666666666666, 'BADGER': -0.016666666666666666, 'BANANA': -0.016666666666666666, 'BIGTIME': -0.016666666666666666, 'BLUR': -0.016666666666666666, 'BLZ': 0.016666666666666666, 'BNB': 0.016666666666666666, 'BSV': 0.016666666666666666, 'BTC': 0.016666666666666666, 'CAKE': -0.016666666666666666, 'CELO': 0.0, 'COMP': 0.0, 'DYDX': 0.0, 'ENS': -0.016666666666666666, 'ETC': 0.016666666666666666, 'ETH': 0.016666666666666666, 'FET': 0.016666666666666666, 'FIL': 0.0, 'FTM': 0.0, 'FTT': 0.0, 

2024-12-18 15:49:46,242 INFO websocket: Websocket connected


[DEX] Getting Positions.
[DEX] SOL: 18.7416
[DEX] BTC: 18.19422
[DEX] ETH: 18.4675
[DEX] BNB: 18.76419
[DEX] AVAX: 18.4695
[DEX] APE: -18.30183
[DEX] OP: 18.41112
[DEX] WLD: -18.3495
[DEX] MKR: 18.4558
[DEX] AAVE: 17.545
[DEX] SUI: -18.4695
[DEX] INJ: 17.4559
[DEX] BLZ: 18.43101
[DEX] BANANA: -17.38
[DEX] BIGTIME: -18.38198
[DEX] KAS: 18.06774
[DEX] BLUR: -18.4729
[DEX] TIA: -18.22335
[DEX] BSV: 18.20384
[DEX] TON: 54.015
[DEX] ADA: -18.6846
[DEX] MINA: 18.15684
[DEX] GAS: 18.4899
[DEX] FET: 19.0229
[DEX] NEAR: 18.08292
[DEX] ORDI: -18.2652
[DEX] BADGER: -18.55332
[DEX] PYTH: -18.515
[DEX] RUNE: 18.57644
[DEX] SUSHI: -18.47772
[DEX] ILV: -18.25629
[DEX] MAV: -18.32474
[DEX] IMX: 18.2367
[DEX] SUPER: 18.9629
[DEX] GALA: -18.38412
[DEX] ACE: -18.346995
[DEX] CAKE: -18.304
[DEX] ENS: -18.32318
[DEX] ETC: 18.29451
[DEX] UMA: 18.29451
[DEX] REQ: -18.38304
[DEX] AR: 18.42687
[DEX] HBAR: -18.55392
[DEX] LISTA: 18.25765
[DEX] POL: -18.23885
[DEX] XLM: -53.6571
[DEX] SAND: -18.6318
[DEX] ALGO: 

In [7]:
dex.get_bid_ask('XLM')

[DEX] Connected to WebSocket getting Bid/Ask for XLM.
Sent: {"method": "post", "id": 123, "request": {"type": "info", "payload": {"type": "l2Book", "coin": "XLM", "mantissa": null}}}
{'channel': 'post', 'data': {'id': 123, 'response': {'type': 'info', 'payload': {'type': 'l2Book', 'data': {'coin': 'XLM', 'time': 1734554977558, 'levels': [[{'px': '0.39714', 'sz': '12071.0', 'n': 1}, {'px': '0.39672', 'sz': '13890.0', 'n': 1}, {'px': '0.39449', 'sz': '14181.0', 'n': 1}, {'px': '0.39253', 'sz': '71053.0', 'n': 1}, {'px': '0.38994', 'sz': '68577.0', 'n': 1}, {'px': '0.38534', 'sz': '67513.0', 'n': 1}], [{'px': '0.39772', 'sz': '6901.0', 'n': 1}, {'px': '0.39775', 'sz': '11927.0', 'n': 1}, {'px': '0.39781', 'sz': '12938.0', 'n': 1}, {'px': '0.398', 'sz': '11904.0', 'n': 1}, {'px': '0.39862', 'sz': '12449.0', 'n': 1}, {'px': '0.39945', 'sz': '12274.0', 'n': 1}, {'px': '0.4006', 'sz': '12105.0', 'n': 1}, {'px': '0.40212', 'sz': '64822.0', 'n': 1}, {'px': '0.40476', 'sz': '68153.0', 'n': 1}, {

(0.39714, 0.39772)